In [1]:
!pip install pandas openpyxl requests beautifulsoup4

# Extraer 4 filas Excel al Programa

In [1]:
import pandas as pd
from openpyxl import load_workbook

def copiar_filas_celdas_especificas(archivo_excel, hoja_origen, hoja_destino, mapeo_celdas, filtro=None, num_filas=None, espaciado=8):
    """
    Copia múltiples filas de una hoja a celdas específicas en otra hoja.

    Parámetros:
    archivo_excel (str): Ruta del archivo Excel
    hoja_origen (str): Nombre de la hoja de origen
    hoja_destino (str): Nombre de la hoja de destino
    mapeo_celdas (dict): Diccionario que mapea columnas de origen a celdas de destino
    filtro (tuple): (columna, valor) para filtrar filas específicas
    num_filas (int): Número de últimas filas a copiar
    espaciado (int): Número de filas de espacio entre cada bloque de datos
    """
    try:
        # Leer la hoja de origen
        df_origen = pd.read_excel(archivo_excel, sheet_name=hoja_origen)

        # Aplicar filtro si se especifica
        if filtro:
            columna, valor = filtro
            df_filtrado = df_origen[df_origen[columna] == valor]
        else:
            df_filtrado = df_origen

        # Seleccionar las últimas n filas si se especifica
        if num_filas:
            filas_a_copiar = df_filtrado.tail(num_filas)
        else:
            filas_a_copiar = df_filtrado

        # Cargar el archivo con openpyxl
        wb = load_workbook(filename=archivo_excel)
        hoja_dest = wb[hoja_destino]

        # Procesar cada fila
        for idx, fila in enumerate(filas_a_copiar.iterrows()):
            desplazamiento = idx * (espaciado + 18)  # 18 es el rango original (19-2)

            # Copiar cada valor a su celda correspondiente
            for columna, celda_base in mapeo_celdas.items():
                # Calcular nueva posición
                letra = celda_base[0]
                numero = int(celda_base[1:]) + desplazamiento
                nueva_celda = f"{letra}{numero}"

                valor = fila[1][columna]
                hoja_dest[nueva_celda] = valor

        # Guardar los cambios
        wb.save(archivo_excel)
        print(f"Valores copiados exitosamente para {len(filas_a_copiar)} filas")

    except Exception as e:
        print(f"Error al copiar los valores: {str(e)}")





# Define el mapeo entre columnas de origen y celdas de destino
mapeo_celdas = {
    'SEMANA': 'B2',      # El valor de la columna 'Nombre' irá a la celda B2
    'PRESIDENCIA': 'O2',        # El valor de la columna 'Edad' irá a la celda D7
    'ORACIÓN': 'O3',
    'TESOROS DE LA BIBLIA': 'O5',
    'BUSQUEMOS PERLAS ESCONDIDAS': 'O6',
    'LECTURA DE LA BIBLIA': 'O7',
    'SMM ASIG 1 ESTUD' : 'L9',
    'SMM ASIG 1 ACOMP' : 'O9',
    'SMM ASIG 2 ESTUD' : 'L10',
    'SMM ASIG 2 ACOMP' : 'O10',
    'SMM ASIG 3 ESTUD' : 'L11',
    'SMM ASIG 3 ACOMP' : 'O11',
    'SMM ASIG 4 ESTUD' : 'L12',
    'SMM ASIG 4 ACOMP' : 'O12',
    'NVC PARTE 1' : 'O15',
    'NVC PARTE 2' : 'O16',
    'ESUDIO LIBRO' : 'O17',
    'LECTOR LIBRO': 'O18',
    'ORACIÓN FINAL': 'O19'
}

# Ejemplo de uso para filtrar por mes

copiar_filas_celdas_especificas(
    archivo_excel="Programación VMC_Septiembre-2024-2025.xlsx",
    hoja_origen="BD ASIG",
    hoja_destino="Formato",
    mapeo_celdas=mapeo_celdas,
    filtro=("MES", "JUNIO")
)


# Ejemplo de uso para últimas 4 filas
"""
copiar_filas_celdas_especificas(
    archivo_excel="Programación VMC_Septiembre-2024-2025.xlsx",
    hoja_origen="BD ASIG",
    hoja_destino="Formato",
    mapeo_celdas=mapeo_celdas,
    num_filas=4
)
"""

Valores copiados exitosamente para 5 filas


'\ncopiar_filas_celdas_especificas(\n    archivo_excel="Programación VMC_Septiembre-2024-2025.xlsx",\n    hoja_origen="BD ASIG",\n    hoja_destino="Formato",\n    mapeo_celdas=mapeo_celdas,\n    num_filas=4\n)\n'

# Extracción de Datos de Web wol.jw.org - Semanas Reunión

In [1]:
# ========================================
# SCRIPT 1: EXTRACTOR DE INFORMACIÓN WOL
# ========================================
# INSTRUCCIONES: Solo necesitas cambiar las URLs de abajo por las semanas que quieres procesar
# Ejemplo de URL: "https://wol.jw.org/es/wol/meetings/r4/lp-s/2024/11/3"

URLS_A_PROCESAR = [
    "https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/23",
    "https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/24",
    "https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/25",
    "https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/26",
    "https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/27"
    # AGREGAR MÁS URLs AQUÍ - UNA POR LÍNEA
    # "https://wol.jw.org/es/wol/meetings/r4/lp-s/2024/12/2",
    # "https://wol.jw.org/es/wol/meetings/r4/lp-s/2024/12/3",
]

# ========================================
# CÓDIGO PRINCIPAL - NO MODIFICAR
# ========================================

import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
import re

def extraer_informacion(url):
    """
    Extrae información de una página de WOL de reunión de entre semana.
    
    Parámetros:
    url (str): URL de la página WOL
    
    Retorna:
    dict: Diccionario con la información extraída
    """
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Inicializar el diccionario de información
        info = {
            "semana": "",
            "libro": "",
            "cancion_inicial": "",
            "tesoros": "",
            "segunda_cancion": "",
            "tercera_cancion": "",
            "titulos_maestros": [],
            "nvc_titulo1": "",
            "nvc_titulo2": "", 
            "nvc_titulo3": ""
        }
        
        # 1. EXTRAER SEMANA - Buscar h1 con id='p1'
        semana_elem = soup.find('h1', id='p1')
        if semana_elem:
            info["semana"] = semana_elem.get_text(strip=True)
        
        # 2. EXTRAER LIBRO DE ESTUDIO - Buscar h2 con id='p2'
        libro_elem = soup.find('h2', id='p2')
        if libro_elem:
            strong_tag = libro_elem.find('strong')
            if strong_tag:
                info["libro"] = strong_tag.get_text(strip=True)
            else:
                info["libro"] = libro_elem.get_text(strip=True)
        
        # 3. EXTRAER CANCIONES
        # Primera canción - h3 con id='p3'
        cancion_elem = soup.find('h3', id='p3')
        if cancion_elem:
            cancion_text = cancion_elem.get_text(strip=True)
            match = re.search(r'Canción\s+(\d+)', cancion_text)
            if match:
                info["cancion_inicial"] = f"Canción {match.group(1)}"
        
        # Extraer todas las canciones restantes
        todas_h3 = soup.find_all('h3')
        canciones_encontradas = []
        cancion_inicial_texto = cancion_elem.get_text(strip=True) if cancion_elem else ""
        
        for h3 in todas_h3:
            texto = h3.get_text(strip=True)
            if "Canción" in texto and texto != cancion_inicial_texto:
                match = re.search(r'Canción\s+(\d+)', texto)
                if match:
                    cancion_formateada = f"Canción {match.group(1)}"
                    if cancion_formateada not in canciones_encontradas:
                        canciones_encontradas.append(cancion_formateada)
        
        # Asignar segunda y tercera canción
        if len(canciones_encontradas) >= 1:
            info["segunda_cancion"] = canciones_encontradas[0]
        if len(canciones_encontradas) >= 2:
            info["tercera_cancion"] = canciones_encontradas[1]
        
        # 4. EXTRAER TESOROS DE LA BIBLIA - Buscar h5, h4 o h3 con id='p5'
        tesoros_elem = soup.find(['h5', 'h4', 'h3'], id='p5')
        if tesoros_elem:
            info["tesoros"] = tesoros_elem.get_text(strip=True)
        
        # 5. EXTRAER TÍTULOS DE "SEAMOS MEJORES MAESTROS"
        # Buscar el encabezado de la sección
        maestros_element = None
        vida_cristiana_element = None
        
        for h2 in soup.find_all('h2'):
            texto = h2.get_text(strip=True)
            if "SEAMOS MEJORES MAESTROS" in texto:
                maestros_element = h2
            elif "NUESTRA VIDA CRISTIANA" in texto:
                vida_cristiana_element = h2
        
        # Extraer títulos entre "SEAMOS MEJORES MAESTROS" y "NUESTRA VIDA CRISTIANA"
        if maestros_element and vida_cristiana_element:
            # Método basado en IDs
            maestros_id = maestros_element.get('id', '')
            vida_id = vida_cristiana_element.get('id', '')
            
            if maestros_id and vida_id:
                maestros_num = int(maestros_id.replace('p', '')) if maestros_id.replace('p', '').isdigit() else 0
                vida_num = int(vida_id.replace('p', '')) if vida_id.replace('p', '').isdigit() else 1000
                
                # Buscar h3 con IDs entre estos dos valores
                for h3 in soup.find_all('h3'):
                    h3_id = h3.get('id', '')
                    if h3_id and h3_id.replace('p', '').isdigit():
                        h3_num = int(h3_id.replace('p', ''))
                        if maestros_num < h3_num < vida_num and h3.find('strong'):
                            strong_text = h3.find('strong').get_text(strip=True)
                            if not strong_text.startswith("Canción"):
                                info["titulos_maestros"].append(strong_text)
        
        # 6. EXTRAER TÍTULOS DE "NUESTRA VIDA CRISTIANA"
        titulos_nvc = []
        estudio_biblico_texto = None
        
        if vida_cristiana_element:
            vida_id = vida_cristiana_element.get('id', '')
            if vida_id and vida_id.replace('p', '').isdigit():
                vida_num = int(vida_id.replace('p', ''))
                
                # Buscar todos los h3 con ID mayor que el de "NUESTRA VIDA CRISTIANA"
                for h3 in soup.find_all('h3'):
                    h3_id = h3.get('id', '')
                    if h3_id and h3_id.replace('p', '').isdigit():
                        h3_num = int(h3_id.replace('p', ''))
                        
                        if h3_num > vida_num and h3.find('strong'):
                            strong_text = h3.find('strong').get_text(strip=True)
                            
                            # Si encontramos "Estudio bíblico de la congregación"
                            if "Estudio bíblico de la congregación" in strong_text:
                                estudio_biblico_texto = strong_text
                                break  # Terminamos la búsqueda una vez encontrado
                            
                            # Otros títulos que no son canciones
                            elif not strong_text.startswith("Canción"):
                                titulos_nvc.append(strong_text)
        
        # Buscar "Estudio bíblico de la congregación" si no se encontró aún
        if not estudio_biblico_texto:
            for h3 in soup.find_all('h3'):
                if h3.find('strong') and "Estudio bíblico de la congregación" in h3.get_text(strip=True):
                    estudio_biblico_texto = h3.find('strong').get_text(strip=True)
                    break
        
        # Asignar los títulos NVC
        info["nvc_titulo1"] = titulos_nvc[0] if len(titulos_nvc) > 0 else ""
        info["nvc_titulo2"] = titulos_nvc[1] if len(titulos_nvc) > 1 else ""
        info["nvc_titulo3"] = estudio_biblico_texto if estudio_biblico_texto else "Estudio bíblico de la congregación"
        
        return info
        
    except Exception as e:
        print(f"Error al extraer información de {url}: {str(e)}")
        return None

def procesar_urls_wol(urls, archivo_principal="Programación VMC_Septiembre-2024-2025.xlsx"):
    """
    Procesa una lista de URLs de WOL y guarda la información en Excel.
    Intenta usar el archivo principal, si no puede, crea un archivo de respaldo.
    
    Parámetros:
    urls (list): Lista de URLs a procesar
    archivo_principal (str): Nombre del archivo Excel principal donde intentar guardar
    """
    datos = []
    archivo_respaldo = "datos_extraidos_wol_jw_org.xlsx"
    archivo_usado = None
    
    print("🔍 Extrayendo información de las páginas WOL...")
    print("-" * 50)
    
    # Extraer información de cada URL
    for i, url in enumerate(urls, 1):
        try:
            print(f"📄 ({i}/{len(urls)}) Procesando: {url}")
            info = extraer_informacion(url)
            
            if info is None:
                print("   ❌ No se pudo extraer información")
                continue
            
            # Información básica para el DataFrame
            datos_basicos = {
                "Semana": info["semana"],
                "Libro": info["libro"],
                "Canción Inicial": info["cancion_inicial"],
                "Tesoros de la Biblia": info["tesoros"],
                "Segunda Canción": info["segunda_cancion"], 
                "Tercera Canción": info["tercera_cancion"]
            }
            
            # Añadir los títulos de maestros como columnas separadas
            for j, titulo in enumerate(info["titulos_maestros"], 1):
                if j <= 4:  # Máximo 4 títulos
                    datos_basicos[f"Maestros Título {j}"] = titulo
            
            # Rellenar títulos de maestros faltantes con cadena vacía
            for j in range(len(info["titulos_maestros"]) + 1, 5):
                datos_basicos[f"Maestros Título {j}"] = ""
            
            # Añadir los títulos NVC
            datos_basicos["NVC Título 1"] = info["nvc_titulo1"]
            datos_basicos["NVC Título 2"] = info["nvc_titulo2"]
            datos_basicos["NVC Título 3"] = info["nvc_titulo3"]
            
            datos.append(datos_basicos)
            print(f"   ✅ Semana extraída: {info['semana']}")
            
            # Mostrar información detallada para verificación
            print("   📋 Información extraída:")
            print(f"      • Libro: {info['libro']}")
            print(f"      • Canciones: {info['cancion_inicial']} | {info['segunda_cancion']} | {info['tercera_cancion']}")
            if info['tesoros']:
                tesoros_corto = info['tesoros'][:50] + "..." if len(info['tesoros']) > 50 else info['tesoros']
                print(f"      • Tesoros: {tesoros_corto}")
            print(f"      • Maestros ({len(info['titulos_maestros'])}): {', '.join(info['titulos_maestros'])}")
            print(f"      • NVC: {info['nvc_titulo1']} | {info['nvc_titulo2']} | {info['nvc_titulo3']}")
            print()
            
        except Exception as e:
            print(f"   ❌ Error al procesar: {str(e)}")
        
        time.sleep(1)  # Pausa entre solicitudes
    
    print("-" * 50)
    
    # Guardar los datos en Excel
    if datos:
        df = pd.DataFrame(datos)
        
        # Intentar guardar en el archivo principal primero
        try:
            print(f"💾 Intentando guardar en archivo principal: {archivo_principal}")
            
            # Verificar si el archivo existe
            try:
                with pd.ExcelWriter(archivo_principal, mode='a', if_sheet_exists='replace', engine='openpyxl') as writer:
                    df.to_excel(writer, sheet_name='Info-reunión', index=False)
                archivo_usado = archivo_principal
                print(f"   ✅ Datos guardados exitosamente en {archivo_principal}")
                
            except FileNotFoundError:
                print(f"   ⚠️  El archivo {archivo_principal} no existe, intentando crearlo...")
                with pd.ExcelWriter(archivo_principal, engine='openpyxl') as writer:
                    df.to_excel(writer, sheet_name='Info-reunión', index=False)
                archivo_usado = archivo_principal
                print(f"   ✅ Archivo {archivo_principal} creado exitosamente")
                
        except Exception as e:
            print(f"   ❌ No se pudo usar el archivo principal: {str(e)}")
            print(f"💾 Intentando crear archivo de respaldo: {archivo_respaldo}")
            
            try:
                with pd.ExcelWriter(archivo_respaldo, engine='openpyxl') as writer:
                    df.to_excel(writer, sheet_name='Info-reunión', index=False)
                archivo_usado = archivo_respaldo
                print(f"   ✅ Datos guardados en archivo de respaldo: {archivo_respaldo}")
                
            except Exception as e2:
                print(f"   ❌ Error crítico: No se pudo guardar en ningún archivo: {str(e2)}")
                return None, None
    
    return datos, archivo_usado

# ========================================
# EJECUCIÓN AUTOMÁTICA
# ========================================
if __name__ == "__main__":
    print("="*60)
    print("🔄 EXTRACTOR DE INFORMACIÓN WOL - VMC")
    print("="*60)
    print(f"📋 Se procesarán {len(URLS_A_PROCESAR)} páginas...")
    print()
    
    # Intentar usar el archivo principal primero
    archivo_principal = "Programación VMC_Septiembre-2024-2025.xlsx"
    
    # Procesar las URLs y extraer información
    datos_procesados, archivo_usado = procesar_urls_wol(URLS_A_PROCESAR, archivo_principal)
    
    if datos_procesados:
        print("\n" + "="*60)
        print("✅ INFORMACIÓN EXTRAÍDA CORRECTAMENTE")
        print("="*60)
        print(f"📊 Total de semanas procesadas: {len(datos_procesados)}")
        print(f"📁 Archivo generado: {archivo_usado}")
        print(f"📋 Hoja de datos: 'Info-reunión'")
        print()
        print("🔍 SIGUIENTE PASO:")
        print("   1. Abre el archivo Excel generado")
        print("   2. Revisa que la información sea correcta")
        print("   3. Complementa o corrige si es necesario")
        print("   4. Ejecuta el script de copia para transferir a la programación")
        print("\n🎉 ¡EXTRACCIÓN COMPLETADA EXITOSAMENTE!")
    else:
        print("\n❌ No se pudo procesar ninguna URL")
    
    print("\n" + "="*60)
    input("Presiona ENTER para cerrar...")

🔄 EXTRACTOR DE INFORMACIÓN WOL - VMC
📋 Se procesarán 5 páginas...

🔍 Extrayendo información de las páginas WOL...
--------------------------------------------------
📄 (1/5) Procesando: https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/23
   ✅ Semana extraída: 2-8 DE JUNIO
   📋 Información extraída:
      • Libro: PROVERBIOS 16
      • Canciones: Canción 36 | Canción 32 | Canción 68
      • Tesoros: 1. Tres preguntas que lo ayudarán a tomar buenas d...
      • Maestros (3): 4. Empiece conversaciones, 5. Haga revisitas, 6. Discurso
      • NVC: 7. Necesidades de la congregación |  | 8. Estudio bíblico de la congregación

📄 (2/5) Procesando: https://wol.jw.org/es/wol/meetings/r4/lp-s/2025/24
   ✅ Semana extraída: 9-15 DE JUNIO
   📋 Información extraída:
      • Libro: PROVERBIOS 17
      • Canciones: Canción 157 | Canción 113 | Canción 35
      • Tesoros: 1. Cómo tener paz en su matrimonio
      • Maestros (3): 4. Empiece conversaciones, 5. Empiece conversaciones, 6. Discurso
      • NVC: 7

# Copiar informacón de la reunión

In [3]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.cell.cell import MergedCell

def copiar_info_reunion(archivo_excel, hoja_origen="Info-reunión", hoja_destino="Formato", espaciado=8):
    """
    Copia la información de reuniones de la hoja Info-reunión a celdas específicas en la hoja Formato,
    manejando correctamente las celdas combinadas.

    Parámetros:
    archivo_excel (str): Ruta del archivo Excel
    hoja_origen (str): Nombre de la hoja que contiene la información de reuniones
    hoja_destino (str): Nombre de la hoja donde se copiará la información
    espaciado (int): Número de filas de espacio entre cada bloque de datos
    """
    try:
        # Leer la hoja de origen
        df_origen = pd.read_excel(archivo_excel, sheet_name=hoja_origen)
        
        # Mostrar las columnas disponibles para verificación
        print("📋 Columnas disponibles en la hoja origen:")
        for i, col in enumerate(df_origen.columns, 1):
            print(f"   {i}. {col}")
        print()

        # Cargar el archivo con openpyxl
        wb = load_workbook(filename=archivo_excel)
        hoja_dest = wb[hoja_destino]

        # Definir el mapeo de columnas a celdas - CORREGIDO
        mapeo_celdas = {
            'Semana': 'B2',                    # Semana
            'Libro': 'I2',                     # Libro de estudio
            'Canción Inicial': 'D3',           # Primera canción
            'Tesoros de la Biblia': 'C5',      # Tesoros
            'Segunda Canción': 'D14',          # Segunda canción
            'Tercera Canción': 'D19',          # Tercera canción (CORREGIDO: era 'Canción Final')
            'Maestros Título 1': 'C9',         # Asignación de maestros 1
            'Maestros Título 2': 'C10',        # Asignación de maestros 2
            'Maestros Título 3': 'C11',        # Asignación de maestros 3
            'Maestros Título 4': 'C12',        # Asignación de maestros 4 (añadido)
            "NVC Título 1": "C15",             # Asignación de vida cristiana 1
            "NVC Título 2": "C16",             # Asignación de vida cristiana 2
            "NVC Título 3": "C17",             # Asignación de vida cristiana 3
        }

        def encontrar_celda_principal(hoja, celda_combinada):
            """
            Encuentra la celda principal de un rango de celdas combinadas.
            """
            for rango in hoja.merged_cells.ranges:
                if celda_combinada.coordinate in rango:
                    return hoja.cell(row=rango.min_row, column=rango.min_col)
            return celda_combinada

        # Procesar cada fila
        filas_procesadas = 0
        for idx, fila in enumerate(df_origen.iterrows()):
            desplazamiento = idx * (espaciado + 18)
            print(f"🔄 Procesando semana {idx + 1}: {fila[1].get('Semana', 'Sin semana')}")

            # Copiar cada valor a su celda correspondiente
            for columna, celda_base in mapeo_celdas.items():
                # Verificar si la columna existe en los datos
                if columna not in df_origen.columns:
                    print(f"   ⚠️  Columna '{columna}' no encontrada, omitiendo...")
                    continue
                
                # Calcular nueva posición
                letra = celda_base[0]
                numero = int(celda_base[1:]) + desplazamiento
                nueva_celda = f"{letra}{numero}"

                celda = hoja_dest[nueva_celda]
                valor = fila[1][columna]

                # Solo copiar si el valor no está vacío
                if pd.notna(valor) and str(valor).strip() != "":
                    # Si es una celda combinada, encontrar la celda principal
                    if isinstance(celda, MergedCell):
                        celda_principal = encontrar_celda_principal(hoja_dest, celda)
                        if celda_principal:
                            celda_principal.value = valor
                    else:
                        celda.value = valor
                    
                    print(f"   ✅ {columna} → {nueva_celda}: {str(valor)[:50]}{'...' if len(str(valor)) > 50 else ''}")
                else:
                    print(f"   ⚪ {columna} → {nueva_celda}: (vacío)")
            
            filas_procesadas += 1
            print(f"   ✅ Semana {idx + 1} completada\n")

        # Guardar los cambios
        wb.save(archivo_excel)
        print(f"🎉 Información copiada exitosamente para {filas_procesadas} semanas")
        print(f"📁 Archivo guardado: {archivo_excel}")

    except Exception as e:
        print(f"❌ Error al copiar la información: {str(e)}")
        print(f"   Tipo de error: {type(e).__name__}")
        
        # Información adicional para debugging
        if 'df_origen' in locals():
            print(f"   Filas en origen: {len(df_origen)}")
            print(f"   Columnas encontradas: {list(df_origen.columns)}")
    
    finally:
        if 'wb' in locals():
            wb.close()

def verificar_datos_origen(archivo_excel, hoja_origen="Info-reunión"):
    """
    Función auxiliar para verificar qué datos están disponibles en la hoja origen.
    """
    try:
        df = pd.read_excel(archivo_excel, sheet_name=hoja_origen)
        print("🔍 VERIFICACIÓN DE DATOS ORIGEN")
        print("=" * 50)
        print(f"📊 Total de filas: {len(df)}")
        print(f"📋 Total de columnas: {len(df.columns)}")
        print("\n📝 Columnas disponibles:")
        for i, col in enumerate(df.columns, 1):
            print(f"   {i:2d}. {col}")
        
        print("\n📄 Primeras 3 filas de datos:")
        print(df.head(3).to_string(max_cols=None, max_colwidth=30))
        print("=" * 50)
        
    except Exception as e:
        print(f"❌ Error al verificar datos: {str(e)}")

# CÓDIGO PRINCIPAL - EJECUCIÓN
if __name__ == "__main__":
    print("🚀 INICIANDO COPIA DE INFORMACIÓN DE REUNIÓN")
    print("=" * 60)
    
    archivo = "Programación VMC_Septiembre-2024-2025.xlsx"
    
    # Paso 1: Verificar datos disponibles
    print("📋 PASO 1: Verificando datos disponibles...")
    verificar_datos_origen(archivo)
    
    # Paso 2: Copiar información
    print("\n🔄 PASO 2: Copiando información...")
    copiar_info_reunion(archivo)
    
    print("\n✅ PROCESO COMPLETADO")
    print("=" * 60)

Error al copiar la información: 'Canción Final'


NameError: name 'urls' is not defined